In [1]:
import numpy as np
import healpy as hp

In [2]:
sky_data_savepath = '/home/zmart/radcos/zvis_sim/test_sky_stokes_data.npz'
sky_data_npz = np.load(sky_data_savepath)

stokesI = sky_data_npz['I']
stokesQ = sky_data_npz['Q']
stokesU = sky_data_npz['U']
stokesV = sky_data_npz['V']

In [3]:
cDD = stokesI + stokesQ
cDH = stokesU + 1.j * stokesV
cHD = np.conjugate(cDH)
cHH = stokesI - stokesQ

In [15]:
npix = cDD.shape[0]
nside = hp.npix2nside(npix)
pixindex1 = np.arange(npix)

In [5]:
coher_tensor = np.array([[cDD,cHD],[cDH,cHH]])

In [6]:
coher_tensor.shape

(2, 2, 49152)

In [96]:
def t_hat_cart(t,p):
    """ Calculate the theta_hat vector at a given point (t,p) """
    thx = np.cos(t)*np.cos(p)
    thy = np.cos(t)*np.sin(p)
    thz = -np.sin(t)
    return np.stack((thx,thy,thz))

def p_hat_cart(t,p):
    """ Calculate the phi_hat vector at a given point (t,p) """
    phx = -np.sin(p)#/np.cos(t)
    phy = np.cos(p)#/np.cos(t)
    phz = np.zeros_like(p)
    return np.stack((phx,phy,phz))

def r_hat_cart(t,p):
    """ Calculate the r_hat vector at a given point (t,p) """
    rx = np.sin(t)*np.cos(p)
    ry = np.sin(t)*np.sin(p)
    rz = np.cos(t)
    return np.stack((rx,ry,rz))
def vrot(vector,theta,phi,rot):
    """ 
        Given a 3-D unit vector specifying the vector field at (theta,phi),
        rotate this field by rot and return the new vector at the new point 
    """
    R = hp.Rotator(rot=rot)
    rvector = R(vector)
    rtheta,rphi = R(theta,phi)
    return (rvector,rtheta,rphi)

We can define the $\cos$ and $\sin$ of the angle between orthogonal bases by: 
\begin{equation}
\begin{array}{lll}
\cos\chi & = & \hat{\theta} \cdot \hat{\theta}' \\
\sin\chi & = & \hat{\theta} \cdot \hat{\phi}' \\
\end{array}
\end{equation}

In [99]:
rot = [0,126]

In [100]:
theta1,phi1 = hp.pix2ang(nside,pixindex1)

In [101]:
rotation_op = hp.Rotator(rot=rot)

In [102]:
# A set of coordinate points to mess around with
angA = (np.arange(6)+1.)*np.pi/12.
angB = (np.arange(6)+1.)*np.pi/12.

In [103]:
# The image points in the rotated coordinate system
angAR, angBR = rotation_op(angA,angB)

In [104]:
# The spherical basis vectors of the starting coordinate system
# projected to cartesian coordinates
t_cart = t_hat_cart(angA,angB)
p_cart = p_hat_cart(angA,angB)

In [105]:
t_cartR = rotation_op(t_cart)
p_cartR = rotation_op(p_cart)

In [106]:
ttcos = np.einsum('a...,a...',t_cart,t_cartR)
tpsin = np.einsum('a...,a...',t_cart,p_cartR)

In [107]:
ppcos = np.einsum('a...,a...',p_cart,p_cartR)
ptsin = np.einsum('a...,a...',p_cart,t_cartR)

In [108]:
print ttcos
print tpsin
print ptsin
print ppcos

[-0.48854867 -0.29007552 -0.19083894 -0.29007552 -0.48854867 -0.58778525]
[ 0.32922683  0.39316522  0.15685836 -0.26299715 -0.65208587 -0.80901699]
[ 0.43761456  0.79767372  0.96587536  0.95052834  0.8575604   0.80901699]
[ 0.89363856  0.60305369  0.20610737 -0.19083894 -0.48142381 -0.58778525]


In [109]:
detrot = ttcos*ppcos -tpsin*ptsin
print detrot

[-0.58066039 -0.48854867 -0.19083894  0.30534395  0.79440198  1.        ]


In [112]:
print (np.sin(angAR)**2.)/(np.sin(angA)**2.)

[ 6.07701396  1.04605934  0.65475574  0.77999601  0.93719173  1.        ]
